# Workflow

打开excl菌属表格，从上往下，查每个菌属（g没有给出，p,c,o,f等均清楚给出了才需要查，但如果f给出的是unidentified这个前缀的，不需要查，如果表格中的数值全部小于0.005（0.5%），也不需要查）

1. 将OTUs.tax-assignments.txt文件拖至excl表格打开，根据第一列文件名排序（方便后面查找），将OTUs.fasta拖至文本文件打开。（打开一个新的excl或txt文件，然后点这两个文件图标，拖到excl或txt工作面板就行）

2. 根据菌属表格从上往下，依次找到需要查找的菌属，复制文件名，去OTUs.tax-assignments.txt文件中查找，有时候查找功能不一定能够找到，需要手动去找，所以一开始要排序。在OTUs.tax-assignments.txt中，确定你要找的菌种对应的OTU数，有时候一个菌属对应了很多OTU数，每个都需要查。

3. 根据OTU数，在OTUs.fasta文件中找到该OTU数的基因序列，复制该基因序列。

4. 打开网址 [http://rdp.cme.msu.edu/seqmatch/seqmatch_intro.jsp](http://rdp.cme.msu.edu/seqmatch/seqmatch_intro.jsp) ，将基因序列复制到文本框中，同时勾选查找方式，具体如图，然后submit。

![method_pic1](./img/screenshot1.png)
![method_pic2](./img/screenshot2.png)

5. 选择show printer friendly results。

![method_pic3](./img/screenshot3.png)

6. 这就是查找的最终结果，找到与你找的那个菌属phylum class order family完全对应的，然后再看相似度，橘红色背景的那个，如果有0.95以上的，把他的菌属名，就是Genus后面的名字，复制到菌属名excel对应的那一栏后面。全部查完都没有0.95以上的，就备注other。


In [1]:
import pandas as pd
import numpy as np
import os
import re
import json
import requests
from bs4 import BeautifulSoup

## 读取菌属表格文件

In [2]:
strain_df = pd.read_excel('./dataset/CA IA/strain.xlsx')
print(len(strain_df))
strain_df.head()

296


,CA1S,CA1B,CA2S,CA2B,IA1S48,IA1B48,IA2S48,IA2B48
count,%,%,%,%,%,%,%,NaN
k__Bacteria;p__Actinobacteria;c__Acidimicrobiia;o__Actinomarinales;f__;g__,0.29,0.059,0.076,0.057,0.032,0.007,0.025,0.004
k__Bacteria;p__Proteobacteria;c__Alphaproteobacteria;o__Rhodobacterales;f__Rhodobacteraceae;Other,0.103,0.091,0.068,0.068,0.109,0.024,0.037,0.029
k__Bacteria;p__Proteobacteria;c__Alphaproteobacteria;o__Rhodospirillales;f__unidentified_Rhodospirillales;g__Defluviicoccus,0.094,0.156,0.222,0.137,0.202,0.112,0.265,0.243
k__Bacteria;p__Proteobacteria;c__Alphaproteobacteria;o__Rhizobiales;f__Rhizobiaceae;Other,0.067,0.071,0.022,0.025,0.099,0.022,0.009,0.009


## 找出需要查的菌属

In [3]:
def needSearch(strain_item):
    """
        判断菌属是否需要查
        返回值：(need_search, code)
            need_search: True/False 如果需要查则返回True,反之是False
            code: 0/1/2/3 标识码
                0 需要查
                1 菌属信息不全
                2 f 中含有 unidentified
                3 数值全部小于0.005（0.5%）
    """
    # 解析菌属字段
    strain_name = strain_item.name
    strain_values = strain_item.values

    # 将菌属字符串分割成为k,p,c,o,f,g的列表
    strain_detail = strain_name.split(';')
    
    need_search = True
    code = 0
    # 检查是否有菌属信息
    for detail in strain_detail[:-1]:
        if detail[-1] == '_' or 'Other' in detail:
            need_search = False
            code = 1
            break

    # 检查 f 是否有 ‘unidentified’
    if 'unidentified' in strain_detail[4]:
        need_search = False
        code = 2

    # 检查数值是否小于 0.5%
    if True not in (strain_values > 0.005):
        need_search = False
        code = 3
    
    return need_search, code

needSearch(strain_df.iloc[1])

(False, 1)

In [4]:
def strainNeedSearch(strain_df):
    """
        返回所有需要查找的菌属
    """
    
    strain_need_search = []
    for i in range(1, len(strain_df)):
        strain_item = strain_df.iloc[i]
        if needSearch(strain_item)[0] == True:
            # 将菌属名存入 strain_name
#             strain_name = ';'.join(strain_item.name.split(';')[:-1])
            strain_name = strain_item.name
            strain_need_search.append(strain_name)
    print("共需要查 %d 个菌属" %(len(strain_need_search)))
    return strain_need_search

strain_need_search = strainNeedSearch(strain_df)

共需要查 38 个菌属


In [5]:
strain_need_search[:5]

['k__Bacteria;p__Proteobacteria;c__Alphaproteobacteria;o__Rhodobacterales;f__Rhodobacteraceae;Other',
 'k__Bacteria;p__Proteobacteria;c__Alphaproteobacteria;o__Rhizobiales;f__Rhizobiaceae;Other',
 'k__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__unidentified_Gammaproteobacteria;f__Competibacteraceae;g__Candidatus_Competibacter',
 'k__Bacteria;p__Bacteroidetes;c__Chlorobia;o__Chlorobiales;f__Chlorobiaceae;g__Prosthecochloris',
 'k__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__unidentified_Gammaproteobacteria;f__Nitrosomonadaceae;g__Nitrosomonas']

## 通过`OTUs.tax_assignments.txt`查找OTU数

### 打开`OTUs.tax_assignments.txt`并读取信息

In [3]:
with open('./dataset/CA IA/OTUs.tax_assignments.txt', 'r') as f:
    OTUs_content = f.read()
print(OTUs_content[:100])

OTU_325	k__Bacteria;p__Actinobacteria;c__unidentified_Actinobacteria;o__Propionibacteriales;f__Nocar


### 找出`strain_need_search`所对应的`OTUs_need_search`

`OTUs_need_search`是一个字典：
```python
OTUs_need_search = {
        "菌属名0":[对应的OTU列表],
        "菌属名1":[对应的OTU列表],
        ...
}

```

In [7]:
def OTUsNeedSearch(strain_need_search):
    """
        根据 strain_need_search 找出对应的 OTUs，
        并存入 OTUs_need_search 字典中
    """
    OTUs_need_search = dict()

    for strain_name in strain_need_search:
        # 正则表达式找出菌属名所对应的 OTUs_content
        info = r'\n([^\n]+%s[^\n]+)\n' %(';'.join(strain_name.split(';')[:-1]))
        OTUs_content_need_search = re.findall(info, OTUs_content)

        # 从 OTUs_content 中获取 OTUs 列表
        OTUs_list_need_search = [content.split('\t')[0] for content in OTUs_content_need_search]

        # 将 OTUs 列表存入 OTUs_need_search 字典中
        OTUs_need_search[strain_name] = OTUs_list_need_search
    
    return OTUs_need_search

OTUs_need_search = OTUsNeedSearch(strain_need_search)

In [8]:
len(OTUs_need_search)

38

## 通过`OTUs.fasta`查找`OTUs_need_search`对应的基因序列

### 打开`OTUs.fasta`并读取信息

In [9]:
with open('./dataset/CA IA/OTUs.fasta', 'r') as f:
    fasta_content = f.read()
fasta_content[:100]

'>OTU_1\nTACGGAAGGGGCTAGCGTTGTTCGGATTTACTGGGCGTAAAGAGCACGTAGGCGGCTTATCAAGTGAGGCGTGAAAGGCC\nTGGGCTCAACCT'

### 找出`OTUs_need_search`所对应的`fasta_need_search`

与`OTUs_need_search`类似，`fasta_need_search`也是一个字典：
```python
fasta_need_search = {
        "菌属名0":[对应的fasta列表],
        "菌属名1":[对应的fasta列表],
        ...
}

```

In [10]:
def fastaNeedSearch(OTUs_need_search):
    """
        根据 OTUs_need_search 找出对应的基因序列，
        并存入 fasta_need_search 字典中
    """
    fasta_need_search = dict()
    
    for strain_name, OTUs_list in OTUs_need_search.items():
        # 正则表达式找出 OTU 所对应的 fasta_content
        fasta_content_need_search = []
        for OTU in OTUs_list:
            info = r'(%s[^>]+)>' %(OTU)
            fasta_content_need_search.append(re.findall(info, fasta_content)[0])
        
        # 从 fasta_content 中获取基因序列
        fasta_list_need_search = [''.join(content.split('\n')[1:]) for content in fasta_content_need_search]
        
        # 将基因序列存入 fasta_need_search 字典中
        fasta_need_search[strain_name] = fasta_list_need_search
    
    return fasta_need_search

fasta_need_search = fastaNeedSearch(OTUs_need_search)

In [11]:
len(fasta_need_search)

38

## 保存`OTUs_need_search`和`fasta_need_search`至文件

In [ ]:
def saveOTUsNeedSearch(OTUs_need_search):
    with open('./dataset/CA IA/OTUs_need_search.txt','w') as outfile:
        json.dump(OTUs_need_search, outfile)

def saveFastaNeedSearch(fasta_need_search):
    with open('./dataset/CA IA/fasta_need_search.txt','w') as outfile:
        json.dump(fasta_need_search, outfile)       
        
saveOTUsNeedSearch(OTUs_need_search)
saveFastaNeedSearch(fasta_need_search)

## 搜索基因序列并获取 Sequence ID

In [2]:
def searchSeqID(seq, threshold=0.95):
	"""搜索基因序列并返回匹配的 sequence ID 
		params:
			seq: (str) 基因序列
			threshold: (float) 搜索的基因匹配程度,范围[0, 1],
		return:
			(array) 
			[[seq_id1, score1],
			 [seq_id2, score2],
			 ...,
			 [seq_idn, scoren]]
	"""
	sess = requests.Session()
	headers = {
		"User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36"
	}
	sess.headers.update(headers)
	try:
		ip = random_choose_ip(ip_list)
	except:
		ip_list = get_ip_list()
		ip = random_choose_ip(ip_list)
	proxies = {
		ip.split(':')[0]:ip,
	}
	data = {
		'printParams': 'no',
		'fromDB': 'no',
		'sequence_file': '(binary)',
		'sequence': "%s" %(seq),
		'strain': 'both',
		'source': 'isolates',
		'size': 'gt1200',
		'quality': 'good',
		'taxonomy': 'rdpHome',
		'num': 20,
		'submit': 'Submit',
	}
	url = "http://rdp.cme.msu.edu/seqmatch/SeqmatchControllerServlet/start"
	res = sess.post(url, data=data, proxies=proxies)
	print("已发送数据", res, ip)

	# 必须 Status --> Summary --> Result 的顺序依次访问，否则可能会报错
	# Status
	res = sess.get("http://rdp.cme.msu.edu/seqmatch/seqmatch_status.jsp?qvector=12&depth=0&currentRoot=0&num=20",
					proxies=proxies)
	print("已获取 Status 页面")
	# Summary
	res = sess.get("http://rdp.cme.msu.edu/seqmatch/seqmatch_sum.jsp?qvector=12&depth=0&currentRoot=0&num=20",
					proxies=proxies)
	print("已获取 Summaray 页面")
	# Result
	res = sess.get('http://rdp.cme.msu.edu/seqmatch/seqmatch_result.jsp?qvector=204&depth=0&currentRoot=0&num=20',
				    proxies=proxies)
	html = res.content.decode()
	print("已获取序列比对数据", res)
	
	# 搜索 match_score 大于等于0.95的序列id
	soup = BeautifulSoup(html, 'html.parser')
	detail_html = soup.find('div', {'class':'details'})
	seq_id_list = [item.text for item in detail_html.find_all('a', {'target':'_blank'})]
	match_score = np.array([float(item.text) for item in detail_html.find_all('span', {'style':'background-color: #FFDBB8'})[1:]])
	seq_id_searched = []
	if True not in (match_score >= threshold):
		print('match_score 全部小于 %.2f' %(threshold))
	for i, score in enumerate(match_score):
		if score >= threshold:
			seq_id_searched.append([seq_id_list[i], score])
	return seq_id_searched

## 多线程获取 Sequence ID

In [5]:
from queue import Queue
from threading import Thread

In [6]:
def searchBatchSeqID(strain_name, fasta_list, queue):
    batch_data = (strain_name, [])
    for i, seq in enumerate(fasta_list):
        try:
            batch_data[1].extend(searchSeqID(seq))
        except:
            batch_data[1].extend([])
    queue.put(batch_data)
    print("\n成功获取 %s 的所有 seq_id" %(strain_name))

In [7]:
def searchAllSeqID(fasta_need_search):
    queue = Queue()
    thds = []
    for strain_name, fasta_list in fasta_need_search.items():
        print(strain_name)
        thd = Thread(target=searchBatchSeqID,
                    args=(strain_name, fasta_list, queue))
        thd.start()
        thds.append(thd)
        print("开启 %s 的线程" %(strain_name))
    for thd in thds:
        thd.join()
        
    seq_id_searched = []
    for thd in thds:
        seq_id_searched += queue.get()
    print('\n成功获取所有seq_id')
    return seq_id_searched